Link Doc Plotly: https://plotly.com/python/


In [ ]:
# Importando as bibliotecas:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as py
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
url = "https://raw.githubusercontent.com/imdevskp/covid_19_jhu_data_web_scrap_and_cleaning/master/covid_19_clean_complete.csv"
df1 = pd.read_csv(url, sep=',', parse_dates=['Date'])    

In [ ]:
df = df1.copy()
df.head(10)

In [ ]:
#informações do dataset
df.info()

In [ ]:
#quantidade de dados
df.shape

In [ ]:
#criando uma coluna casos ativos
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

In [ ]:
#visualizando as 10 primeiras linhas do dataset
df.tail(10)

In [ ]:
#substituindo nome no dataset
df['Country/Region'] = df['Country/Region'].replace('Mainland China', 'China')

In [ ]:
#verificando dados faltantes
df.isnull().sum()

In [ ]:
#preenchendo valores vazios
df[['Province/State']] = df[['Province/State']].fillna('')
df[['Confirmed', 'Deaths', 'Recovered', 'Active']] = df[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)

In [ ]:
df.tail(10)

In [ ]:
#descrição dados temporais
df.Date.describe()

In [ ]:
#ferificando a frequancia de ocorrencia de uma data especifica
df.loc[df['Date'] == '2020-04-09'].sum()

In [ ]:
#!pip install dash==0.21.1
#!pip install dash-core-components==0.23.0
#!pip install dash-html-components==0.11.0
#!pip install dash-renderer==0.13.0
#!pip install plotly==2.7.0

In [ ]:
#Agrupando dados
df_agrup = df.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
df_agrup.tail(5)

In [ ]:
#pie_chart = go.Pie(labels=df['Country/Region'].value_counts().index, values=df['Confirmed'].value_counts())
#py.iplot([pie_chart])

In [ ]:
#Ordenando o dataset
df_agrup.sort_values(by='Confirmed', ascending=False)

In [ ]:
# Obtém o número de casos confirmados, mortes, recuperados e ativos agrupando por região ordem alfabetica
df_group_paises = df.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
df_group_paises

In [ ]:
# Ordena por países com mais casos confirmados
df_group_paises.sort_values(by='Confirmed', ascending=False)

In [ ]:
#agrupando quantidade de casos recuperados, mortes e ativos por data
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()

In [ ]:
temp.tail()

In [ ]:
# melt do pandas para remodelar o dataframe com variável e valor, para ter a quantidade de casos recuperados, mortos e ativos.
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')

In [ ]:
#temp.head(10)
temp.tail(10)

In [ ]:
# Habilita modo offline
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
from plotly import graph_objs as go
# Definindo o renderizador:
import plotly.io as pio
pio.renderers
pio.renderers.default = "colab"

In [ ]:
# Cores
recuperados = '#21bf73'
mortes = '#ff2e63'
ativos = '#fe9801'

In [ ]:
# Criando um gráfico de área
import plotly.express as px
fig = px.area(temp, 
              x="Date", 
              y="Count", 
              color='Case', 
              height=600,
              title='Casos ao longo do tempo',
              color_discrete_sequence = [recuperados, mortes, ativos])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
fig = px.choropleth(df_agrup,                                          # casos agrupados por país
                    locations="Country/Region",                             # definindo as regiões no mapa
                    locationmode='country names',# define o modo de localização para todas regiões
                    color=np.log(df_agrup["Confirmed"]),# define a cor pelo o valor de casos confirmados (aplica o log)
                    hover_name='Country/Region',   # define o texto interativo com o nome da região
                    hover_data=["Confirmed", "Deaths"], # define o texto interativo com o numero de casos confirmasos e mortes
                    animation_frame=df_agrup["Date"].dt.strftime('%d-%m-%Y'),   # define o animate_frame com as datas
                    title='Casos ao longo do tempo',         # define título
                    color_continuous_scale=px.colors.sequential.Magenta)    # define a paleta de cores
fig.update_layout(autosize=False, width=1200, height=800)  # define tamanho da figura
fig.show()

In [ ]:
#mortes por tempo
fig = px.choropleth(df_agrup,                       # casos agrupados por país
                    locations="Country/Region",     # definindo as regiões no mapa
                    locationmode='country names',   # define o modo de localização para todas regiões
                    color=np.log(df_agrup["Deaths"]),  # define a cor pelo o valor de casos confirmados (aplica o log)
                    hover_name='Country/Region',       # define o texto interativo com o nome da região
                    hover_data=["Confirmed", "Deaths"],# define o texto interativo com o numero de casos confirmasos e mortes
                    animation_frame=df_agrup["Date"].dt.strftime('%d-%m-%Y'),   # define o animate_frame com as datas
                    title='Mortes ao longo do tempo',  # define título
                    color_continuous_scale=px.colors.sequential.Magenta)   # define a paleta de cores
fig.update_layout(autosize=False, width=1200, height=800)
fig.show()

In [ ]:
#!pip install plotly==4.5.2
#!pip install --upgrade plotly

In [ ]:
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
# Filtra os dados considerando o último dia da base de dados
completo = df[df['Date'] == max(df['Date'])]

In [ ]:
completo.head()

In [ ]:
# Plota painel        
        
fig = px.treemap(completo.sort_values(by='Confirmed', ascending=False).reset_index(drop=True), 
                 path=["Country/Region", "Province/State"],
                 values="Confirmed",
                 height=600,
                 title='Número de Casos Confirmados',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()  
# Plota Painel
fig = px.treemap(completo.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country/Region", "Province/State"],
                 values="Deaths",
                 height=600,
                 title='Número de Mortes Confirmadas',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
fig = px.line(df_agrup,
              x="Date",
              y="Confirmed",
              color='Country/Region',
              height=600,
              title='Casos Confirmados',
              color_discrete_sequence = px.colors.qualitative.Dark2 )
fig.show()

fig = px.line(df_agrup,
              x="Date",
              y="Deaths",
              color='Country/Region',
              height=600,
              title='Mortes Confirmadas',
              color_discrete_sequence = px.colors.qualitative.Dark2)
fig.show()


In [ ]:
# Obtem os dados do último dia da base de dados
temp = df[df['Date'] == max(df['Date'])]

import folium
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=1)

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Lat'], temp.iloc[i]['Long']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(temp.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Deaths']),
        radius=int(temp.iloc[i]['Confirmed'])**1.1).add_to(m)
m